In [1]:
import pandas as pd
from glob import glob
import numpy as np
from tqdm import tqdm

In [2]:
path = "data/tickers/spx_group_sector.csv"
df = pd.read_csv(path)
df = df.loc[df.field == "INDUSTRY_SECTOR"].reset_index(drop=True)
df.loc[:, "ticker"] = df.ticker.map(lambda x: x.replace("/", " "))

## Find problematic tickers (tickers with gaps)

In [38]:
st

100%|██████████| 1103/1103 [04:28<00:00,  4.11it/s]


In [ ]:
ticker_paths = glob('data/tickers/spx/*.csv')

tickers_to_skip = []
for p in tqdm(ticker_paths):
    stock_name = p.split('/')[3]
    stock = p.split('/')[3].split(' ')[0]
    exchange = p.split('/')[3].split(' ')[1]

    stock_df = pd.read_csv(p)
    if stock not in tickers_to_skip and stock_df.shape[0] > 2:
        stock_df = stock_df.drop([0,1,2], 0)
        stock_df.rename(columns={'ticker': 'date'}, inplace=True)
        stock_df.index = pd.to_datetime(stock_df['date'])
        stock_df = stock_df.drop('date', 1)
        stock_df[stock_df.columns[0]] = stock_df[stock_df.columns[0]].astype(float)
        stock_df = stock_df.resample('D').mean()

        stock_df['month'] = [dt_ref.month for dt_ref in stock_df.index]
        stock_df['year'] = [dt_ref.year for dt_ref in stock_df.index]
        stock_df['count'] = [1 if pd.isna(val) else 0 for val in stock_df[stock_df.columns[0]]]

        group_stock_df = stock_df.groupby(['month', 'year']).sum()['count'].reset_index()
        group_stock_df.index = group_stock_df['year']
        group_stock_df = group_stock_df.sort_index() 

        median = group_stock_df['count'].median() 
        std = group_stock_df['count'].std()
        if np.sum(np.where(group_stock_df['count'] > (median+ 1 * std), 1, 0)) > 10 and len(df.loc[(df['stock'] == stock)&(df['exchange'] != exchange)]['ticker']) >= 2:
            new_stock = df.loc[(df['stock'] == stock)&(df['exchange'] != exchange)]['ticker'].iloc[0]
            new_stock_df = pd.read_csv(os.path.join('data', 'tickers', 'spx', new_stock + '.csv'))
            if new_stock_df.shape[0] > 2:
                new_stock_df = new_stock_df.drop([0,1,2], 0)
                new_stock_df.rename(columns={'ticker': 'date'}, inplace=True)
                new_stock_df.index = pd.to_datetime(new_stock_df['date'])
                new_stock_df = new_stock_df.drop('date', 1)
                new_stock_df[new_stock_df.columns[0]] = new_stock_df[new_stock_df.columns[0]].astype(float)
                new_stock_df = new_stock_df.resample('D').mean()

                adj_stock_df = pd.concat([stock_df.drop(['month', 'year', 'count'], 1), new_stock_df], axis=1)
                adj_stock_df[adj_stock_df.columns[1]] = [row[adj_stock_df.columns[0]] if pd.isna(row[adj_stock_df.columns[1]]) else row[adj_stock_df.columns[1]] for i, row in adj_stock_df.iterrows()]
                adj_stock_df = adj_stock_df.drop(adj_stock_df.columns[0], 1)
                adj_stock_df.to_csv(os.path.join('data', 'tickers', 'spx_adj', new_stock + '.csv'))
                tickers_to_skip.append(stock)
            else:
                stock_df.drop(['month', 'year', 'count'], 1).to_csv(os.path.join('data', 'tickers', 'spx_adj', stock_name + '.csv'))   
                tickers_to_skip.append(stock)
        else:
            stock_df.drop(['month', 'year', 'count'], 1).to_csv(os.path.join('data', 'tickers', 'spx_adj', stock_name + '.csv'))
            tickers_to_skip.append(stock)

